# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd

df = pd.read_csv("homepage_actions.csv")
print(df.shape)
print(df.head())

print("Unique users:", df["id"].nunique())

viewers = set(df[df["action"] == "view"]["id"])
clickers = set(df[df["action"] == "click"]["id"])

print("Users who both viewed and clicked:", len(viewers & clickers))
print("Users who clicked but did not view:", len(clickers - viewers))

control_users = set(df[df["group"] == "control"]["id"])
experiment_users = set(df[df["group"] == "experiment"]["id"])

print("Users in both control & experiment groups:", len(control_users & experiment_users))


(8188, 4)
                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
Unique users: 6328
Users who both viewed and clicked: 1860
Users who clicked but did not view: 0
Users in both control & experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [2]:
#Your code here
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

control_clicks = df[(df["group"] == "control") & (df["action"] == "click")]["id"].nunique()
experiment_clicks = df[(df["group"] == "experiment") & (df["action"] == "click")]["id"].nunique()

control_total = df[df["group"] == "control"]["id"].nunique()
experiment_total = df[df["group"] == "experiment"]["id"].nunique()

count = np.array([experiment_clicks, control_clicks])
nobs = np.array([experiment_total, control_total])

z_stat, p_val = proportions_ztest(count, nobs)
print("Z-statistic:", z_stat)
print("p-value:", p_val)


Z-statistic: 2.618563885349469
p-value: 0.008830075576595804


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [3]:
#Your code here
control_ctr = control_clicks / control_total
expected_experiment_clicks = experiment_total * control_ctr
print("Expected experiment clicks (if same rate as control):", expected_experiment_clicks)
print("Actual experiment clicks:", experiment_clicks)


Expected experiment clicks (if same rate as control): 838.0168067226891
Actual experiment clicks: 928


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [4]:
#Your code here
std_dev = np.sqrt(experiment_total * control_ctr * (1 - control_ctr))
z_score = (experiment_clicks - expected_experiment_clicks) / std_dev
print("Z-score:", z_score)


Z-score: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [5]:
#Your code here
from scipy.stats import norm

p_value = 1 - norm.cdf(z_score)
print("P-value:", p_value)
from scipy.stats import norm

p_value = 1 - norm.cdf(z_score)
print("P-value:", p_value)


P-value: 0.00012486528006949715
P-value: 0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**
The results from this verification method closely match the outcome of the earlier two-proportion z-test. Both approaches yield a statistically significant result, with a very small p-value (< 0.05), indicating that the experimental homepage led to a higher click-through rate compared to the control. Therefore, we can confidently conclude that the experimental homepage was more effective.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.